# Berend's code test for point cloud processing

In [13]:
import sys
import os
import glob

import gc
import queue

import glob
import psutil
from psutil import virtual_memory

import pandas as pd
import numpy as np
import scipy
from scipy import spatial

import math
from math import ceil

import time
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor
import pickle

import laspy
from laspy.file import File

sys.path.insert(0, "D:/GitHub/eEcoLiDAR/myPhD_escience_analysis/from_msc_students/")

import pcfeatures_v4_forcomparetest
from pcfeatures_v4_forcomparetest import par_calc

import lidar_funcs_v2
from lidar_funcs_v2 import hasNumbers

In [14]:
# global variables

workingdirectory="D:/Koma/escience/test_escience_2018Nov/exampledata/"
filename="tile_00022"

In [20]:
if __name__ == "__main__":

    ########### Define import las file ###########	

    input_file_path = workingdirectory+filename+".las"
        
    print("Memory mappping", os.path.basename(input_file_path), "...")
        
        #in_LAS = File(input_file_path[i], mode='r')
    in_LAS = File(input_file_path,mode='r')
        
        #Read out the size and therefore point density by len points divided by minx,y max xy distances.
    input_rows = 'max'

    if input_rows == "max" :
            #input_rows = None
        input_rows = int(len(in_LAS))
    else:
        input_rows = int(input_rows)


    ########### Explore LiDAR dataset ########### 
	
    print("Determining average point density of the input file")
        #Top left x coordinate
    left_x = np.min(in_LAS.x)
    top_y = np.min(in_LAS.y[np.where(in_LAS.x == left_x)])
        
    top_left_xy = np.array((left_x,top_y)) #used
    bot_left_xy = np.array((left_x,top_y+10))# used
    top_right_xy = np.array((left_x+10,top_y)) #used
    bot_right_xy = np.array((left_x+10,top_y+10)) #unused

    A = np.array((top_left_xy,bot_left_xy,top_left_xy,top_right_xy))
    B = np.array((top_right_xy,bot_right_xy,bot_left_xy,bot_right_xy))
        
    dd = np.sqrt(np.einsum('ij,ij->i',A-B,A-B))
        
    surf_area = dd[0] * dd[2]
        
    x_range = np.where(np.logical_and(in_LAS.x > top_left_xy[0], in_LAS.x < top_right_xy[0]))
        #y_range = np.where(np.logical_and(in_LAS.y > top_left_xy[1], in_LAS.y < bot_left_xy[1]))
        
    x_range_points = np.vstack(np.array([in_LAS.x[x_range],in_LAS.y[x_range]])).transpose()
        
        
    num_points = len(np.where(np.logical_and(x_range_points[:,1] > top_left_xy[1], x_range_points[:,1] < bot_left_xy[1]))[0])
        
        
    est_p_density = num_points/surf_area
        
    print("Estimated point density of this file is {} points/m2".format(est_p_density))
	
    fullstarttime = time.time()

	########### Define features ###########
    
    features = ['eig1', 'eig2','max_z', 'min_z','mean_z',
                'median_z','var_z','skew_z','std_z',"kurto_z","coeffvar_z"]
    		
	########### Build up KDTree ###########
    
    print("Defining constants..")

    x = None
  
    #k = 5 0.5pt/m2
    #Callibration: K = 50 for 16 pt/m2
    #50 / 16 = 3.125 K's per point
        
    k = int(3.125 * est_p_density)
        
    #introduce a minimum K check (We need atleast 3 to do anything sensible!)
        
    if k < 3:
        k = 3
            
    print("Using K={}".format(k))
        
    print("Retrieving points to be processed from memory")
      
    points = np.vstack([in_LAS.x[0:input_rows],in_LAS.y[0:input_rows],in_LAS.z[0:input_rows]]).transpose()
    
        
    points_remain_prop = pd.DataFrame(np.vstack([in_LAS.intensity[0:input_rows],in_LAS.return_num[0:input_rows],in_LAS.num_returns[0:input_rows]]).transpose(),columns = ["intensity","return_number","number_of_returns"])
        #points_remain_prop = np.vstack([in_LAS.intensity[0:input_rows]]).transpose()
  
    print("Constructing KDTree")

    #evaluate options for quicker kdtree building
    tree=scipy.spatial.cKDTree(points,balanced_tree=0,compact_nodes=0)
    point_cloud_prop = pd.DataFrame([], columns=features)
	
	########### Set CPU usage ###########
	
    print("Multiprocessing")
 
    ##Dynamic chunk size based upon CPU/MEMORY ratio
    #500.000 seems to be good for 40 cores with 256 gb of ram.
    #256 GB / 40 = 6.4 GB/Core. 500.000 chunk size for K = 50 seems to be appropriate.
    #500.000 * 50 = 25.000.000 points per 6.4 GB/Core
    #25.000.000 / 6.4 = 3.906.205 points / GB/core
    #Values have been experimentally found.
	
    cpu_count = mp.cpu_count()
    mem=virtual_memory()
    print(mem)
    mem_tot = mem.free/1000000000
    print(mem_tot)
    cpu_mem_ratio = mem_tot/cpu_count
    print(cpu_mem_ratio)
    #K = 50
    #Apparently classifying points copy huge datasets to the memory (duplicates) as such a new variable H is introduced which is a integer between 0 - 1 to be able to scale.
	
    H = 0.5
    tot_points_cpu = (3906205*H) * cpu_mem_ratio
    print(tot_points_cpu)
    chunk = int(tot_points_cpu / k)
 
    print('Using chunk size:', chunk) 

    print("Total processors (hyper threading) available: ",mp.cpu_count(),". Using: ",cpu_count, "threads")
    
    #Maximum amount of points to go through
    
    num_points = len(points)

    #Number of points per CPU
    
    points_p_cpu = int(num_points/cpu_count)
    index_order = np.zeros((cpu_count,2),dtype=int)
    
    #Create an index work order per core
    
    print("Create start stop indexes for n_cpu_cores-1")
    
    for l in range(0,cpu_count):
    
        index_start = int(l * (points_p_cpu))
    
        index_order[l] = [int(index_start),int(index_start + (points_p_cpu))]
    
        if l == cpu_count-1:
    
            index_order[-1,1] = index_order[-1,1] + num_points%cpu_count              #Add remaining index to last cpu core

    print(index_order) 

    #Split the work in cpu_cores, let the function run with those parameters.
       
    print("Splitting work load across n_cpu-1")
    
    classification = False
    tensor_filter = False
    
    q = queue.Queue()
    
	########### Parallel computation ###########
	
    mod_start = time.time()
        
    with ThreadPoolExecutor(max_workers=cpu_count) as executor:
    
        futures = []
    
        for j in range(0,cpu_count):
    
            start = index_order[j,0]
    
            stop = index_order[j,1]

            print("Submitting thread", j)
            futures.append(executor.submit(par_calc, start, stop, points,points_remain_prop,tree,chunk,k,features,tensor_filter,classification))
    
           
    print(time.time()-mod_start)
        
    print("Done!")
       
    print("Retrieving results")
    mod_start = time.time()

    output = []
        
    for future in futures:
  
        output = future.result()
            #print(output)
        point_cloud_prop = pd.concat([point_cloud_prop,output],axis=0)
            #print(point_cloud_prop)
        
    print(time.time()-mod_start)
    print("Done!")
	
    fullendtime = time.time()
    fulldifftime=fullendtime - fullstarttime

    print(fulldifftime)
        
    del output
    
    print("Calculations are done, just finishing up here..")
    
    print("Keeping track of parameters used...")
    
        
 	########### Organizing the columns for output ###########
	
    print("Bundeling everything together..")
        
    points = pd.DataFrame(points,columns = ["x","y","z"])
    points_remain_prop = pd.DataFrame(points_remain_prop,columns = ["intensity","return_number","number_of_returns"])
        
    points = points.ix[point_cloud_prop.index]
    points_remain_prop = points_remain_prop.ix[point_cloud_prop.index]
    print(points_remain_prop.columns)
                
    point_cloud = points
    del points
    point_cloud = pd.concat([point_cloud,points_remain_prop],axis=1)
    del points_remain_prop
    point_cloud = pd.concat([point_cloud,point_cloud_prop],axis=1)
    del point_cloud_prop
        
    print(point_cloud.columns)
    print("removing duplicate columns")
    point_cloud = point_cloud.loc[:,~point_cloud.columns.duplicated()]
	
    #point_cloud.to_csv(input_file_path[:-4]+'_ascii.csv',sep=',',index=False,header=True)

	########### Output ###########
	
    print("Outputting to las file")
    
    out_filename = workingdirectory+filename+"_inBerendcode2.las"
    
    print(out_filename)
    out_LAS = File(out_filename, mode = "w", header = in_LAS.header)      
        
    out_LAS.define_new_dimension(name="eig1"+"_"+str(k), data_type=9, description= "Spatial feature")
    out_LAS.define_new_dimension(name="eig2"+"_"+str(k), data_type=9, description= "Spatial feature")
    #out_LAS.define_new_dimension(name="eig3"+"_"+str(k), data_type=9, description= "Spatial feature")
    out_LAS.define_new_dimension(name="max_z"+"_"+str(k), data_type=9, description= "Spatial feature")
    out_LAS.define_new_dimension(name="min_z"+"_"+str(k), data_type=9, description= "Spatial feature")
    out_LAS.define_new_dimension(name="mean_z"+"_"+str(k), data_type=9, description= "Spatial feature")
    out_LAS.define_new_dimension(name="median_z"+"_"+str(k), data_type=9, description= "Spatial feature")
    out_LAS.define_new_dimension(name="var_z"+"_"+str(k), data_type=9, description= "Spatial feature")
    out_LAS.define_new_dimension(name="skew_z"+"_"+str(k), data_type=9, description= "Spatial feature")
    out_LAS.define_new_dimension(name="std_z"+"_"+str(k), data_type=9, description= "Spatial feature")
    out_LAS.define_new_dimension(name="kurto_z"+"_"+str(k), data_type=9, description= "Spatial feature")
    out_LAS.define_new_dimension(name="coeffvar_z"+"_"+str(k), data_type=9, description= "Spatial feature")
        
    print(point_cloud.columns)

    out_LAS.x = point_cloud['x']
        #point_cloud.drop('x')
        
    out_LAS.y = point_cloud['y']
    out_LAS.z = point_cloud['z']
    out_LAS.intensity = point_cloud['intensity'] 
    out_LAS.return_num = point_cloud['return_number']
        #print(point_cloud["number_of_returns"])
    point_cloud["number_of_returns"] = point_cloud["number_of_returns"]
        #print(point_cloud["number_of_returns"])
    out_LAS.num_returns = point_cloud['number_of_returns']
        
        #Setting attributes Maybe do this with "try" ?
    setattr(out_LAS,'eig1'+"_"+str(k),point_cloud['eig1'])

    setattr(out_LAS,'eig2'+"_"+str(k),point_cloud['eig2'])

    #setattr(out_LAS,'eig3'+"_"+str(k),point_cloud['eig3'])

    setattr(out_LAS,'max_z'+"_"+str(k),point_cloud['max_z'])

    setattr(out_LAS,'min_z'+"_"+str(k),point_cloud['min_z'])

    setattr(out_LAS,'mean_z'+"_"+str(k),point_cloud['mean_z'])

    setattr(out_LAS,'median_z'+"_"+str(k),point_cloud['median_z'])

    setattr(out_LAS,'var_z'+"_"+str(k),point_cloud['var_z'])

    setattr(out_LAS,'skew_z'+"_"+str(k),point_cloud['skew_z'])

    setattr(out_LAS,'std_z'+"_"+str(k),point_cloud['std_z'])

    setattr(out_LAS,'kurto_z'+"_"+str(k),point_cloud['kurto_z'])

    setattr(out_LAS,'coeffvar_z'+"_"+str(k),point_cloud['coeffvar_z'])
    
    out_LAS.close()
	
    fullendtime_end = time.time()
    fulldifftime_end=fullendtime_end - fullstarttime
	
    print(fulldifftime_end)

Memory mappping tile_00022.las ...
Determining average point density of the input file
Estimated point density of this file is 1.72 points/m2
Defining constants..
Using K=5
Retrieving points to be processed from memory
Constructing KDTree
Multiprocessing
svmem(total=17056247808, available=9674051584, percent=43.3, used=7382196224, free=9674051584)
9.674051584
2.418512896
4723603.58345984
Using chunk size: 944720
Total processors (hyper threading) available:  4 . Using:  4 threads
Create start stop indexes for n_cpu_cores-1
[[      0  927205]
 [ 927205 1854410]
 [1854410 2781615]
 [2781615 3708822]]
Splitting work load across n_cpu-1
Submitting thread 0
Submitting thread 1
Submitting thread 2
Submitting thread 3
RangeIndex(start=1854410, stop=2781615, step=1)RangeIndex(start=2781615, stop=3708822, step=1)

RangeIndex(start=0, stop=927205, step=1)
RangeIndex(start=927205, stop=1854410, step=1)
21.591952323913574
Done!
Retrieving results
0.9374327659606934
Done!
23.729258060455322
Calcula

C:\Program Files\Miniconda3\lib\site-packages\ipykernel_launcher.py:214: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Program Files\Miniconda3\lib\site-packages\ipykernel_launcher.py:215: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


Index(['intensity', 'return_number', 'number_of_returns'], dtype='object')
Index(['x', 'y', 'z', 'intensity', 'return_number', 'number_of_returns',
       'coeffvar_z', 'eig1', 'eig2', 'kurto_z', 'max_z', 'mean_z', 'median_z',
       'min_z', 'planarity', 'skew_z', 'sphericity', 'std_z', 'var_z'],
      dtype='object')
removing duplicate columns
Outputting to las file
D:/Koma/escience/test_escience_2018Nov/exampledata/tile_00022_inBerendcode2.las
Index(['x', 'y', 'z', 'intensity', 'return_number', 'number_of_returns',
       'coeffvar_z', 'eig1', 'eig2', 'kurto_z', 'max_z', 'mean_z', 'median_z',
       'min_z', 'planarity', 'skew_z', 'sphericity', 'std_z', 'var_z'],
      dtype='object')
29.948686122894287
